In [15]:
import pandas as pd
import glob
import pandas_ta as ta

headers = ['Date','Time','Open','High','Low','Close','Volume']
path = "C:\\Users\\desma\\Downloads\\jupyter work\\DATA\\"
#set up so i could run multiple datasets if wanted
filenames = glob.glob(path + "QQQ15m.txt")
for file in filenames:
    print("\nReading file = ", file)
    df = pd.read_csv(file, header = None)
    df.columns=headers
#df['Datetime'] = df['Date'] + ' ' + df['Time']
df.index = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df['VWAP'] = ta.vwap(df.High, df.Low, df.Close, df.Volume)
dfLen = len(df)


Reading file =  C:\Users\desma\Downloads\jupyter work\DATA\QQQ15m.txt


In [16]:
import plotly.graph_objects as go
def plotSignal(df):
    fig = go.Figure(data=[go.Candlestick(x=df.index, open=df['Open'],  high=df['High'], low=df['Low'],
                                         close=df['Close'], name = '15m Candle')])
    fig.add_trace(go.Line(x=df.index, y=df.VWAP, name='VWAP'))
    fig.show()
df1 = df[0:1600]
#plotSignal(df1)

In [24]:
signal = [0]*dfLen
for row in range(1, dfLen):
    prevClose = df.Close[row-1]
    currClose = df.Close[row]
    prevVWAP = df.VWAP[row-1]
    currVWAP = df.VWAP[row]
    #long position --> signal == 1
    if prevClose<prevVWAP and currClose>currVWAP:
        signal[row] = 1
        
    #short position --> signal == 2
    elif prevClose>prevVWAP and currClose<currVWAP:
        signal[row] = 2
        
df['Signal'] = signal
df['Signal'].value_counts()

0    19793
1     1958
2     1923
Name: Signal, dtype: int64

In [25]:
def SIGNAL():
    return df.Signal

In [27]:
from backtesting import Strategy, Backtest
import numpy as np

class myStrat(Strategy):
    def init(self):
        super().init()
        self.signal1=self.I(SIGNAL)     
        
    def next(self):
        super().next()
        trailSL = self.data.Close[-1]*.0025
        for trade in self.trades:
            if trade.is_long:
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - trailSL)
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + trailSL)
        
        if self.signal1==1 and len(self.trades)==0: #Long Signal
            self.buy()
            
        if self.signal1==2 and len(self.trades)==0: #Short Signal
            self.sell()
            
bt = Backtest(df, myStrat, cash=100_000, margin = 1/1, commission=.000)
stat = bt.run()
stat
#bt.plot(show_legend=False, superimpose = True)

Start                     2021-01-04 04:00:00
End                       2022-06-23 11:15:00
Duration                    535 days 07:15:00
Exposure Time [%]                   54.439469
Equity Final [$]                 63479.027195
Equity Peak [$]                    104718.147
Return [%]                         -36.520973
Buy & Hold Return [%]               -9.083096
Return (Ann.) [%]                  -27.787031
Volatility (Ann.) [%]               10.132555
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -42.493242
Avg. Drawdown [%]                   -2.774956
Max. Drawdown Duration      530 days 18:30:00
Avg. Drawdown Duration       31 days 11:30:00
# Trades                                 1658
Win Rate [%]                         35.88661
Best Trade [%]                       2.741532
Worst Trade [%]                     -1.520209
Avg. Trade [%]                    